In [7]:
import pandas as pd
import numpy as np

# Cargar el dataset limpio
df = pd.read_csv("online_retail_limpio.csv")

In [8]:
df 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,TotalPrice,Weekday,Hour,Region,Season,NumPurchases
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12,15.30,Wednesday,8,UK,Winter,34
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,20.34,Wednesday,8,UK,Winter,34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12,22.00,Wednesday,8,UK,Winter,34
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,20.34,Wednesday,8,UK,Winter,34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,20.34,Wednesday,8,UK,Winter,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397879,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,2011-12,10.20,Friday,12,International,Winter,4
397880,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12,12.60,Friday,12,International,Winter,4
397881,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12,16.60,Friday,12,International,Winter,4
397882,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12,16.60,Friday,12,International,Winter,4


In [9]:
# Convierto la columna InvoiceDate a datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Extraigo de nuevo la semana del año
df['Week'] = df['InvoiceDate'].dt.isocalendar().week

# Creo la columna ProductLine desde Description
def clasificar_producto(desc):
    desc = str(desc).upper()
    if any(x in desc for x in ['PAPER', 'NOTEBOOK', 'STATIONERY']):
        return 'Stationery'
    elif any(x in desc for x in ['LANTERN', 'CUSHION', 'PLAQ', 'VASE']):
        return 'Home Decor'
    elif any(x in desc for x in ['MUG', 'TEA', 'PLATE', 'BOWL', 'KITCHEN']):
        return 'Kitchenware'
    elif any(x in desc for x in ['GAME', 'TOY', 'PUZZLE', 'STICKER']):
        return 'Toys & Games'
    elif any(x in desc for x in ['CHRISTMAS', 'EASTER', 'VALENTINE']):
        return 'Seasonal'
    else:
        return 'Others'

df['ProductLine'] = df['Description'].apply(clasificar_producto)

# Creo ahora la columna Territory desde Country
def asignar_territorio(pais):
    pais = str(pais)
    if pais in ['United Kingdom', 'France', 'Germany', 'Spain', 'Portugal', 'Italy', 'Belgium', 'Netherlands']:
        return 'EU'
    elif pais in ['USA', 'Canada']:
        return 'NA'
    elif pais in ['Japan', 'Hong Kong', 'Singapore']:
        return 'Asia'
    else:
        return 'Other'

df['Territory'] = df['Country'].apply(asignar_territorio)

# Agrupar por semana, producto, país y nuevas columnas
df_agrupado = (
    df.groupby(['Week', 'ProductLine', 'Country', 'Territory', 'StockCode', 'Description'])
    .agg(TotalUnits=('Quantity', 'sum'))
    .reset_index()
    .sort_values(by=['Week', 'TotalUnits'], ascending=[True, False])
)

# Añadir columnas vacías para Make.com
df_agrupado['Alerta'] = ''
df_agrupado['Riesgo'] = ''

# Guardar en CSV si deseas exportar
df_agrupado.to_csv("tabla_alertas+riesgo.csv", index=False)